### Get the stock with max weight from a user's portfolio


In [21]:
import pandas as pd
import numpy as np

user_portfolios = pd.read_csv('user_portfolios.csv')

# Group by user_id, then group by sector and sum the weights for each sector then get the sector with the max weight
user_data = user_portfolios.groupby(['user_id', 'sector']).sum().reset_index()
user_data = user_data.loc[user_data.groupby('user_id')['weight'].idxmax()]

# Group by user_id and get the max weight and the stock and sector that is associated with it
#user_data = user_portfolios.groupby('user_id').apply(lambda x: x.loc[x['weight'].idxmax()])



# Save the csv like 'max_weight_user_portfolios.csv'
user_data.to_csv('max_weight_user_portfolios.csv', index=False)



### Get a similarity weight by comparing the max weight of a stock's sector to other people's weight on that sector
For each user in the list, we check the max sector and compare it with all other's sector weight, we will compute a similarity matrix with the % on that sector.

In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Function for similarity weights
def compare_weights(weight1, weight2):
    if weight2 >= weight1:
        return 1.0
    else:
        return weight2 / weight1

# Read the csv files
max_weight_user_portfolios = pd.read_csv('max_weight_user_portfolios.csv')
user_portfolios = pd.read_csv('user_portfolios.csv')

# Initialize the similarity matrix
user_ids = max_weight_user_portfolios['user_id'].unique()
similarity_matrix = pd.DataFrame(index=user_ids, columns=user_ids, dtype=float)

# Iterate through each user in the max_weight_user_portfolios DataFrame with a progress bar
for index, row in tqdm(max_weight_user_portfolios.iterrows(), total=max_weight_user_portfolios.shape[0], desc="Processing users"):
    user1_id = row['user_id']
    user1_sector = row['sector']
    user1_weight = row['weight']
    
    for index2, row2 in tqdm(max_weight_user_portfolios.iterrows(), total=max_weight_user_portfolios.shape[0], desc=f"Comparing {user1_id} with others", leave=False):
        user2_id = row2['user_id']
        
        if user1_id != user2_id:  # Not the same user
            if user1_sector == row2['sector']:
                similarity = 1.0
            else:
                # Find the weight of the sector of user1 in user2's portfolio
                user2_portfolios = user_portfolios[user_portfolios['user_id'] == user2_id]
                user2_sector_weight = user2_portfolios[user2_portfolios['sector'] == user1_sector]['weight']
                
                if not user2_sector_weight.empty:
                    # Sum the weights if there are multiple entries for the same sector
                    user2_sector_weight_value = user2_sector_weight.sum()
                    similarity = compare_weights(user1_weight, user2_sector_weight_value)
                else:
                    similarity = 0.0  # No matching sector found
            
            # Store the similarity in the matrix
            similarity_matrix.loc[user1_id, user2_id] = similarity

# Fill diagonal with 1.0 (each user is perfectly similar to themselves)
np.fill_diagonal(similarity_matrix.values, 1.0)

# Save the similarity matrix to a CSV file
similarity_matrix.to_csv('sector_similarity_matrix.csv')

# Print the similarity matrix
print(similarity_matrix)


Processing users: 100%|██████████| 500/500 [01:00<00:00,  8.30it/s]


          0         1         2         3         4         5         6    \
0    1.000000  0.428208  0.000000  0.000000  0.000000  0.000000  0.000000   
1    0.099481  1.000000  0.230413  0.000000  0.411513  0.088890  0.962862   
2    0.000000  0.041760  1.000000  0.000000  0.000000  0.044785  0.000000   
3    0.496928  0.000000  0.000000  1.000000  1.000000  0.000000  0.068760   
4    0.832452  0.000000  0.000000  1.000000  1.000000  0.000000  0.115186   
..        ...       ...       ...       ...       ...       ...       ...   
495  1.000000  0.336505  0.000000  0.000000  0.000000  0.000000  0.000000   
496  0.000000  0.000000  0.000000  0.112612  0.000000  0.000000  0.000000   
497  0.136921  1.000000  0.317130  0.000000  0.566389  0.122345  1.000000   
498  0.141248  0.547308  0.000000  0.312304  0.301954  1.000000  0.612783   
499  0.000000  0.056161  1.000000  0.000000  0.000000  0.060231  0.000000   

          7         8         9    ...       490       491       492  \
0  

In [38]:
import pandas as pd
from collections import defaultdict

# Load data
user_portfolios = pd.read_csv('user_portfolios.csv')
sector_similarity_matrix = pd.read_csv('sector_similarity_matrix.csv', index_col=0)
sp500_companies = pd.read_csv('constituents.csv')  # Contains columns 'Symbol' and 'GICS Sector'

# Ensure user_id is treated as a string across all dataframes
user_portfolios['user_id'] = user_portfolios['user_id'].astype(str)
sector_similarity_matrix.index = sector_similarity_matrix.index.astype(str)
sector_similarity_matrix.columns = sector_similarity_matrix.columns.astype(str)

# Function to get top sector for a user
def get_top_sector(user_id):
    user_data = user_portfolios[user_portfolios['user_id'] == user_id]
    top_sector = user_data.groupby('sector')['weight'].sum().idxmax()
    return top_sector

# Function to recommend stocks based on top sector and similar users
def recommend_stocks(user_id):
    top_sector = get_top_sector(user_id)
    all_users = sector_similarity_matrix.index
    
    # Aggregate stocks in the top sector from all users
    all_users_data = user_portfolios[user_portfolios['user_id'].isin(all_users)]
    sector_stocks = all_users_data[all_users_data['sector'] == top_sector]
    
    # Calculate weighted similarity scores for each stock
    stock_similarity_scores = defaultdict(float)
    for other_user_id in all_users:
        if other_user_id != user_id:
            similarity_score = sector_similarity_matrix.loc[user_id, other_user_id]
            user_stocks = sector_stocks[sector_stocks['user_id'] == other_user_id]['stock']
            for stock in user_stocks:
                stock_similarity_scores[stock] += similarity_score
    
    # Filter out stocks already owned by the user
    user_stocks = set(user_portfolios[user_portfolios['user_id'] == user_id]['stock'])
    recommended_stocks = {stock: score for stock, score in stock_similarity_scores.items() if stock not in user_stocks}
    
    # Normalize the scores to percentages
    max_score = max(recommended_stocks.values(), default=0)
    if max_score > 0:
        recommended_stocks_percentage = {stock: (score / max_score) * 100 for stock, score in recommended_stocks.items()}
    else:
        recommended_stocks_percentage = recommended_stocks
    
    # Sort recommendations by weighted similarity score and return
    sorted_recommendations = sorted(recommended_stocks_percentage.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations

# Example usage
user_id = '2'
recommendations = recommend_stocks(user_id)
print("Recommended stocks for user:", recommendations)
print("Sector:", get_top_sector(user_id))
print(len(recommendations))

# Check number of stocks in sector 'Energy' in constituents
print(len(sp500_companies[sp500_companies['GICS Sector'] == 'Energy']))


Recommended stocks for user: [('VLO', 100.0), ('EQT', 48.1850408917668), ('MRO', 46.32227655718456), ('KMI', 37.701634965908276), ('HAL', 37.07334197380337), ('DVN', 36.42317505317472), ('OXY', 36.32253862857897), ('COP', 35.944882181765536), ('APA', 34.5666126262307), ('FANG', 29.849304008105715), ('XOM', 29.53154955297092), ('EOG', 25.816799197929143), ('OKE', 25.296967821811457), ('CVX', 24.133112809538957), ('HES', 18.055090863079894), ('BKR', 15.517778822612193), ('PSX', 12.17926996827351), ('SLB', 10.935478137904589), ('WMB', 8.679987262504198)]
Sector: Energy
19
22


## Similarity matrix of risk

In [4]:
import pandas as pd

# Load the data
user_portfolios = pd.read_csv('user_portfolios.csv')
constituents_with_changes = pd.read_csv('constituents_with_changes.csv')

# Ensure user_id is treated as a string
user_portfolios['user_id'] = user_portfolios['user_id'].astype(str)
constituents_with_changes['Symbol'] = constituents_with_changes['Symbol'].astype(str)

# Create a dictionary to store user risk tolerance
user_risk = {}

# Iterate through each user in the user_portfolios
for user_id in user_portfolios['user_id'].unique():
    # Get the user's portfolio
    user_data = user_portfolios[user_portfolios['user_id'] == user_id]
    
    # Calculate the risk tolerance
    total_weighted_move = 0
    total_weight = 0
    for _, row in user_data.iterrows():
        stock = row['stock']
        weight = row['weight']
        last_month_move = constituents_with_changes[constituents_with_changes['Symbol'] == stock]['last_month_move'].values
        if last_month_move:
            total_weighted_move += weight * abs(last_month_move[0])
            total_weight += weight
    
    if total_weight > 0:
        user_risk[user_id] = total_weighted_move / total_weight
    else:
        user_risk[user_id] = None  # Handle users with no valid stocks


print(user_risk)
# Save the user risk tolerance to a CSV file
user_risk_df = pd.DataFrame(user_risk.items(), columns=['user_id', 'risk_tolerance'])
user_risk_df.to_csv('user_risk_tolerance.csv', index=False)


{'0': 0.8124640095144797, '1': 1.3237391148058768, '2': 0.30229442343477386, '3': 0.678328990564775, '4': 0.803948020291824, '5': 1.504326966214126, '6': 0.7924019232515677, '7': 1.687662043886592, '8': 0.9860516794650539, '9': 0.45542712393658974, '10': 0.7438946485814547, '11': 0.3932116276717037, '12': 1.045419721269647, '13': 0.7226224545225648, '14': 0.7570239416332565, '15': 0.6113793225845153, '16': 0.853797569625546, '17': 0.7450912007847847, '18': 1.0764322462000546, '19': 0.3224358056655514, '20': 0.4320234768399458, '21': 1.2880705115989644, '22': 1.361749787980325, '23': 0.5052319859761778, '24': 0.7847765012752527, '25': 1.7669461875272592, '26': 0.55351426551229, '27': 0.6622646977823884, '28': 0.806499028725325, '29': 0.5693083835176388, '30': 1.571146862973419, '31': 0.9868566616539002, '32': 1.6509555365020774, '33': 0.6059211423874117, '34': 1.20167132952972, '35': 0.7849094883712979, '36': 0.5902706428186115, '37': 1.531354909250624, '38': 0.606672378419994, '39': 1.

In [5]:
import pandas as pd
import numpy as np

# Load the user risk tolerance data
user_risk_tolerance = pd.read_csv('user_risk_tolerance.csv')

# Create an empty DataFrame for the risk similarity matrix
user_ids = user_risk_tolerance['user_id']
similarity_matrix = pd.DataFrame(index=user_ids, columns=user_ids, dtype=float)

# Function to compute similarity based on risk tolerance
def compute_similarity(risk1, risk2):
    return 1 - abs(risk1 - risk2)

# Iterate over each pair of users to compute similarity
for i, user1 in enumerate(user_ids):
    risk1 = user_risk_tolerance.loc[user_risk_tolerance['user_id'] == user1, 'risk_tolerance'].values[0]
    for j, user2 in enumerate(user_ids):
        risk2 = user_risk_tolerance.loc[user_risk_tolerance['user_id'] == user2, 'risk_tolerance'].values[0]
        similarity_matrix.at[user1, user2] = compute_similarity(risk1, risk2)

# Normalize the similarity matrix to the range [0, 1]
min_similarity = similarity_matrix.min().min()
max_similarity = similarity_matrix.max().max()
similarity_matrix = (similarity_matrix - min_similarity) / (max_similarity - min_similarity)

# Save the similarity matrix to a CSV file
similarity_matrix.to_csv('risk_similarity_matrix.csv')

print("Risk similarity matrix saved to 'risk_similarity_matrix.csv'.")


Risk similarity matrix saved to 'risk_similarity_matrix.csv'.


In [10]:
import pandas as pd
from collections import defaultdict

# Load the data
user_portfolios = pd.read_csv('user_portfolios.csv')
constituents_with_changes = pd.read_csv('constituents_with_changes.csv')
risk_similarity_matrix = pd.read_csv('risk_similarity_matrix.csv', index_col=0)

# Ensure correct data types
user_portfolios['user_id'] = user_portfolios['user_id'].astype(str)
constituents_with_changes['Symbol'] = constituents_with_changes['Symbol'].astype(str)
risk_similarity_matrix.columns = risk_similarity_matrix.columns.astype(str)
risk_similarity_matrix.index = risk_similarity_matrix.index.astype(str)

def recommend_stocks_by_risk(user_id):
    if user_id not in risk_similarity_matrix.index:
        raise ValueError(f"User ID {user_id} not found in the risk similarity matrix")

    stock_scores = defaultdict(float)

    # Get the list of stocks already owned by the user
    user_owned_stocks = set(user_portfolios[user_portfolios['user_id'] == user_id]['stock'])

    # Iterate through each stock in constituents_with_changes
    for stock in constituents_with_changes['Symbol']:
        if stock in user_owned_stocks:
            continue  # Skip stocks already owned by the user

        # Find users who have this stock in their portfolio
        for _, user_row in user_portfolios[user_portfolios['stock'] == stock].iterrows():
            other_user_id = user_row['user_id']
            weight = user_row['weight']
            
            if other_user_id in risk_similarity_matrix.columns:
                similarity = risk_similarity_matrix.at[user_id, other_user_id]
                if pd.isna(similarity):
                    similarity = 0
                stock_scores[stock] += weight * similarity

    # Normalize the scores to percentages
    if stock_scores:
        max_score = max(stock_scores.values())
        min_score = min(stock_scores.values())
        if max_score > min_score:
            normalized_scores = {stock: 100 * (score - min_score) / (max_score - min_score) for stock, score in stock_scores.items()}
        else:
            normalized_scores = {stock: 100 for stock, score in stock_scores.items()}
    else:
        normalized_scores = {}

    # Sort the recommendations by score in descending order
    sorted_recommendations = sorted(normalized_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations

# Example usage
user_id = '0'
recommendations = recommend_stocks_by_risk(user_id)
print("Recommended stocks for user:", recommendations)


Recommended stocks for user: [('VLO', 100.0), ('AMP', 88.72445814590446), ('KEY', 87.1953581764769), ('NTRS', 86.51465776732981), ('HUBB', 81.58421736286954), ('PKG', 80.72475460725914), ('ED', 79.33196248789348), ('C', 78.20230569527364), ('MTD', 76.24706719692938), ('GE', 74.83990423777304), ('EBAY', 74.64351144932617), ('KHC', 73.35441405642035), ('CDNS', 71.96969648709052), ('TECH', 71.19470899255106), ('BDX', 68.03165558286408), ('LUV', 67.52732293228047), ('BLDR', 67.09444727742704), ('EXC', 66.86129603323512), ('NCLH', 66.42851009139382), ('MCHP', 66.08442327006077), ('EIX', 65.9542343641118), ('TAP', 65.71819549172157), ('FDS', 64.98499312360593), ('MDT', 64.56815148278642), ('VTR', 64.24196858869959), ('WTW', 64.1258962999051), ('BK', 63.44529183840693), ('CMG', 63.36345131026092), ('COR', 63.311788892541756), ('QCOM', 63.22140199976014), ('CMA', 63.16499852162023), ('XYL', 62.49271412778728), ('IQV', 62.16471030369647), ('BA', 61.09452050899065), ('GD', 60.311436183577506), (